Import necessary modules.

In [79]:
import pandas as pd
import json

Helper methods.

In [41]:
cricket_data = pd.read_csv('datacsv.csv')
cricket_data = cricket_data.drop(columns=['Unnamed: 0'], axis=1)
aus_data = (cricket_data[cricket_data['currentInning.team.shortName'] == 'AUS']).sort_values(by=['over'])
scot_data = (cricket_data[cricket_data['currentInning.team.shortName'] == 'SCOT']).sort_values(by=['over'])
print(len(scot_data) + len(aus_data))
print(len(cricket_data))

552
552


In [103]:
def get_player_runs(data):

    # Initialize
    batsmen_runs = pd.DataFrame(columns=['name', 'runs'])
    curr_batsmen_name = ''
    curr_batsmen_runs = 0
    batsmen_runs_dict = {}

    # Fill the data frame
    for batsmen in data['currentBatsmen']:
        batsmen = json.loads(batsmen.replace("\'", "\""))
        curr_batsmen_name = batsmen[0]['name']
        curr_batsmen_runs = batsmen[0]['runs']
        batsmen_runs = batsmen_runs.append({
            'name': curr_batsmen_name,
            'runs': curr_batsmen_runs
        }, ignore_index=True)

    # Get the player names
    player_names = batsmen_runs['name'].unique()

    # Fill the dictionary
    for player in player_names:
        batsmen_runs_dict[player] = ((batsmen_runs[batsmen_runs['name'] == player])['runs']).max()

    # Return the dictionary
    return batsmen_runs_dict

def get_inning_runs(data):

    total_runs = data['currentInning.runs'].max()

    return total_runs


In [105]:
# Player - Runs Australia
player_runs_aus = get_player_runs(aus_data)
total_runs_aus = get_inning_runs(aus_data)
print(player_runs_aus)
print(total_runs_aus)


# Player - Runs Scotland
player_runs_scot = get_player_runs(scot_data)
total_runs_scot = get_inning_runs(scot_data)
print(player_runs_scot)
print(total_runs_scot)


{'Adam Gilchrist': 46, 'Matthew Hayden': 60, 'Ricky Ponting': 113, 'Michael Clarke': 15, 'Brad Hodge': 29, 'Michael Hussey': 4, 'Shane Watson': 18, 'Brad Hogg': 40}
334
{'Fraser Watts': 9, 'Majid Haq': 16, 'Navdeep Poonia': 1, 'Ryan Watson': 6, 'Gavin Hamilton': 3, 'Dougie Brown': 19, 'Colin Smith': 51, 'Craig Wright': 4, 'Glenn Rogers': 6}
131
